In [1]:
# NÃO MEXER NESSA CÉLULA
# Tenta instalar automaticamente se estiver no Colab
try:
    import google.colab
    !pip install skl2onnx onnxmltools pandas scikit-learn
except ImportError:
    print("Rodando localmente... certifique-se de ter instalado: pip install skl2onnx onnxmltools pandas scikit-learn")

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Imports para exportação ONNX
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType
import onnxmltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 94.5 MB/s eta 0:00:00:00:0100:01


In [2]:
# --- 2. CÓDIGO DO TIME ---
# DICA: Mantenha a variável do modelo final com o nome 'pipeline'

print("Simulando treinamento...")

# Dados Dummy (substituir pelo CSV real depois)
data = {'text': ['gostei muito', 'péssimo serviço', 'ótimo', 'ruim'], 'label': [1, 0, 1, 0]}
df = pd.DataFrame(data)

# Pipeline Simples
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

pipeline.fit(df['text'], df['label'])
print("Modelo treinado e salvo na variável 'pipeline'.")

Simulando treinamento...
Modelo treinado e salvo na variável 'pipeline'.


In [5]:
# NÃO ALTERAR NADA DESSA CÉLULA E MANTER COMO ULTIMA
# --- 3. EXPORTAÇÃO HÍBRIDA (Local Inteligente + Download Colab) ---
import os
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import StringTensorType

print("Iniciando conversão para ONNX...")

# 1. Definição do Contrato (Input)
initial_type = [('text_input', StringTensorType([None, 1]))]

try:
    # 2. Converte o modelo (Assumindo que sua variável se chama 'pipeline')
    onnx_model = convert_sklearn(pipeline, initial_types=initial_type)
    arquivo_nome = "sentiment_model.onnx"
    
    # 3. Lógica Híbrida de Salvamento
    # Procura a pasta do Java subindo um nível (../) a partir da pasta data-science
    pasta_java = "../src/main/resources"
    
    if os.path.exists(pasta_java):
        # CENÁRIO A: Rodando no seu PC (Antigravity) ou GitHub Actions
        caminho_final = os.path.join(pasta_java, arquivo_nome)
        with open(caminho_final, "wb") as f:
            f.write(onnx_model.SerializeToString())
        
        print(f"✅ SUCESSO! O arquivo foi salvo automaticamente na pasta do Java:")
        print(f"📂 {os.path.abspath(caminho_final)}")
        print("Agora é só commitar a mudança no git!")
        
    else:
        # CENÁRIO B: Rodando no Google Colab (Nuvem)
        print("⚠️ Pasta do projeto Java não encontrada (Ambiente Colab detectado).")
        print("Salvando arquivo local para download...")
        
        with open(arquivo_nome, "wb") as f:
            f.write(onnx_model.SerializeToString())
            
        # Força o download no navegador
        try:
            from google.colab import files
            files.download(arquivo_nome)
            print("⬇️ Download iniciado no navegador.")
        except ImportError:
            print(f"Arquivo salvo em: {os.path.abspath(arquivo_nome)}")

except NameError:
    print("❌ ERRO: A variável 'pipeline' não foi encontrada. Rode a célula de treino anterior.")
except Exception as e:
    print(f"❌ ERRO NA CONVERSÃO: {e}")

Iniciando conversão para ONNX...
⚠️ Pasta do projeto Java não encontrada (Ambiente Colab detectado).
Salvando arquivo local para download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Download iniciado no navegador.
